<a href="https://colab.research.google.com/github/afifrizkyandika11551100310/IF6082_NLP/blob/main/23521034_NLP_BERT_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementasi BERT dalam Text Classification**

*  IF6082 Natural Language Processing 

## **Task 01 :** 

Import Module dan Data

In [24]:
pip install tensorflow_text

In [25]:
pip install Sastrawi

In [26]:
pip install swifter

In [27]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import numpy as np
import ast
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords

In [28]:
# Data Training 

df_train = pd.read_csv('train.csv', index_col=[0])
df_train

,text_a,label
0,betewe buka twitter cuman ngetweet liat home b...,no
1,mas piyuuu mugo2 corona tuh mulut tersumpal ma...,no
2,e100ss gini buka informasi sejelas nya identit...,yes
3,neng solo wes ono terduga corona cobo neng ati...,no
4,midiahn nii akun gak takut takut nya isu coron...,no
...,...,...
21596,depok panas ga karuan kereta sampe pasming huj...,no
21597,oxfara arie kriting yg lebi goblo nya orang ke...,no
21598,virus corona menyaba depok cuci tangan makan n...,no
21599,mata sipit tinggal depok udah abis dah bahan c...,no


In [29]:
# Data Testing

df_test = pd.read_csv('test.csv')
df_test

,text_a,label
0,jek dajal ga depok bang,no
1,detikcom untung depok masuk wilayah nya ridwan...,no
2,df dom jakarta depok yg gunain vc cabang nya c...,no
3,your2rl depok jkt,no
4,doakan indonesia selamat virus corona pkb depo...,yes
...,...,...
2795,ku tenang2 bae ku sih ya corona nya ga depok k...,no
2796,guru hati hati ya virus corona uda indonesia t...,yes
2797,4 terawan menyebut virus corona indonesia terd...,yes
2798,realffk buhari can t pronounce corona virus,no


## **Task 02 :**

Exploratory Data Analysis 

In [30]:
df_train.groupby('label').describe()

text_a                                        
       count unique                         top freq
label                                               
no     15512  11732        sensus penduduk 2020    4
yes     6089   4617  hoax berjemur lawan corona    5

In [31]:
df_yes = df_train[df_train['label']=='yes']

df_no = df_train[df_train['label']=='no']

print("yes Dataset Shape:", df_yes.shape)

print("no Dataset Shape:", df_no.shape)

yes Dataset Shape: (6089, 2)
no Dataset Shape: (15512, 2)


In [32]:
df_no_downsampled = df_no.sample(df_yes.shape[0])
df_no_downsampled.shape

(6089, 2)

In [33]:
df_train = pd.concat([df_yes , df_no_downsampled])

In [34]:
df_train['label'].value_counts()

yes    6089
no     6089
Name: label, dtype: int64

## **Task 03 :** 

Preprocessing

Preprocessing terdapat beberapa tahap dengan implementasi sebagai berikut

### **3.1 Preprocessing label Data Training dan Testing**

In [35]:
# Preprocessing label data training

df_train['label']=df_train['label'].apply(lambda x: 1 if x=='yes' else 0)
df_train

,text_a,label
2,e100ss gini buka informasi sejelas nya identit...,1
7,jokowi menteri2 nya silakan tes corona,1
8,pencegahan corona other moms minum multivitami...,1
11,kemenkesri mohon yg transparan mudah spt utk d...,1
18,jateng jatim jakarta siaga corona info disebar...,1
...,...,...
2260,gak masuk kerja pala pusing banget berat kek m...,0
10032,melaporkan dokter terawan perbuatan menyenangk...,0
19459,kvmbangsat cnnindonesia menkes sembuh ko g kha...,0
6233,mood edit laptop dihalaman rumah karna ikutin ...,0


In [36]:
# Preprocessing data testing

df_test['label']=df_test['label'].apply(lambda x: 1 if x=='yes' else 0)
df_test

,text_a,label
0,jek dajal ga depok bang,0
1,detikcom untung depok masuk wilayah nya ridwan...,0
2,df dom jakarta depok yg gunain vc cabang nya c...,0
3,your2rl depok jkt,0
4,doakan indonesia selamat virus corona pkb depo...,1
...,...,...
2795,ku tenang2 bae ku sih ya corona nya ga depok k...,0
2796,guru hati hati ya virus corona uda indonesia t...,1
2797,4 terawan menyebut virus corona indonesia terd...,1
2798,realffk buhari can t pronounce corona virus,0


### **3.2 Preprocessing Text Data Training**

In [37]:
# Remove Special Character

replace_puncts = {'`': "'", '′': "'", '“':'"', '”': '"', '‘': "'"}

strip_chars = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '[', ']', '>', '=', '+', '\\', '•',  '~', '@', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

puncts = ['!', '?', '$', '&', '/', '%', '#', '*','£']

def clean_str(x):
    x = str(x)
    
    x = x.lower()
    
    x = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", x)
    
    for k, v in replace_puncts.items():
        x = x.replace(k, f' {v} ')
        
    for punct in strip_chars:
        x = x.replace(punct, ' ') 
    
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
        
    x = x.replace(" '", " ")
    x = x.replace("' ", " ")
        
    return x

df_train['text_a'] = df_train['text_a'].apply(clean_str)
df_train['text_a'].head(10)

2     e100ss gini buka informasi sejelas nya identit...
7                jokowi menteri2 nya silakan tes corona
8     pencegahan corona other moms minum multivitami...
11    kemenkesri mohon yg transparan mudah spt utk d...
18    jateng jatim jakarta siaga corona info disebar...
22             prabndriapsari corona penulara nya cepat
27    dhonipusing rumah sakit daftar peserta simulas...
29    jimanah dikriminalisasi aja nih virus corona m...
31    skrinews dr pingkan pijoh mphm dk satgas covid...
32       tangkal corona prancis tutup toko restoran bar
Name: text_a, dtype: object

In [38]:
# Apply Case Folding

df_train['text_a'] = [line.lower() for line in df_train['text_a']]
df_train['text_a'].head(10)

2     e100ss gini buka informasi sejelas nya identit...
7                jokowi menteri2 nya silakan tes corona
8     pencegahan corona other moms minum multivitami...
11    kemenkesri mohon yg transparan mudah spt utk d...
18    jateng jatim jakarta siaga corona info disebar...
22             prabndriapsari corona penulara nya cepat
27    dhonipusing rumah sakit daftar peserta simulas...
29    jimanah dikriminalisasi aja nih virus corona m...
31    skrinews dr pingkan pijoh mphm dk satgas covid...
32       tangkal corona prancis tutup toko restoran bar
Name: text_a, dtype: object

In [39]:
## Stopword Removal

nltk.download('stopwords')
stop = stopwords.words('indonesian')

with tf.device('/device:GPU:0'):  
  df_train['text_a'].apply(lambda x: [item for item in x if item not in stop])
print(df_train['text_a'].head(10))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2     e100ss gini buka informasi sejelas nya identit...
7                jokowi menteri2 nya silakan tes corona
8     pencegahan corona other moms minum multivitami...
11    kemenkesri mohon yg transparan mudah spt utk d...
18    jateng jatim jakarta siaga corona info disebar...
22             prabndriapsari corona penulara nya cepat
27    dhonipusing rumah sakit daftar peserta simulas...
29    jimanah dikriminalisasi aja nih virus corona m...
31    skrinews dr pingkan pijoh mphm dk satgas covid...
32       tangkal corona prancis tutup toko restoran bar
Name: text_a, dtype: object


In [40]:
## Tokenozation

df_train['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in df_train['text_a']] 
print(df_train['tokenized_text'].head(10))

2     [ss, gini, buka, informasi, sejelas, nya, iden...
7          [jokowi, menteri, nya, silakan, tes, corona]
8     [pencegahan, corona, other, moms, minum, multi...
11    [kemenkesri, mohon, yg, transparan, mudah, spt...
18    [jateng, jatim, jakarta, siaga, corona, info, ...
22       [prabndriapsari, corona, penulara, nya, cepat]
27    [dhonipusing, rumah, sakit, daftar, peserta, s...
29    [jimanah, dikriminalisasi, aja, nih, virus, co...
31    [skrinews, dr, pingkan, pijoh, mphm, dk, satga...
32    [tangkal, corona, prancis, tutup, toko, restor...
Name: tokenized_text, dtype: object


In [41]:
# Data Testing 
df_test['text_a'] = df_test['text_a'].apply(clean_str)

In [42]:
df_test['text_a'] = [line.lower() for line in df_test['text_a']]

In [43]:
with tf.device('/device:GPU:0'):  
  df_test['text_a'].apply(lambda x: [item for item in x if item not in stop])

In [44]:
df_test['text_a'] = [simple_preprocess(line, deacc=True) for line in df_test['text_a']]

### **3.3 Text Normalization Data Training**

In [47]:
df_train.head(2)

,text_a,label,tokenized_text
2,e100ss gini buka informasi sejelas nya identit...,1,"[ss, gini, buka, informasi, sejelas, nya, iden..."
7,jokowi menteri2 nya silakan tes corona,1,"[jokowi, menteri, nya, silakan, tes, corona]"


In [48]:
kamus_normalisasi = pd.read_csv('kamus_normalisasi.csv')
kamus_normalisasi

,kata_tidak_baku,kata_baku
0,krn,karena
1,sampe,sampai
2,serem,seram
3,met,selamat
4,dri,dari
...,...,...
15445,gataunya,enggak taunya
15446,gtau,enggak tau
15447,gatau,enggak tau
15448,fans2,fan-fan


In [49]:
normalizad_dict = {}

def normalized_term(document):
    return [normalizad_dict[term] if term in normalizad_dict else term for term in document]

df_train['text_normal'] = df_train['tokenized_text'].apply(normalized_term)

In [50]:
df_train.head()

,text_a,label,tokenized_text,text_normal
2,e100ss gini buka informasi sejelas nya identit...,1,"[ss, gini, buka, informasi, sejelas, nya, iden...","[ss, gini, buka, informasi, sejelas, nya, iden..."
7,jokowi menteri2 nya silakan tes corona,1,"[jokowi, menteri, nya, silakan, tes, corona]","[jokowi, menteri, nya, silakan, tes, corona]"
8,pencegahan corona other moms minum multivitami...,1,"[pencegahan, corona, other, moms, minum, multi...","[pencegahan, corona, other, moms, minum, multi..."
11,kemenkesri mohon yg transparan mudah spt utk d...,1,"[kemenkesri, mohon, yg, transparan, mudah, spt...","[kemenkesri, mohon, yg, transparan, mudah, spt..."
18,jateng jatim jakarta siaga corona info disebar...,1,"[jateng, jatim, jakarta, siaga, corona, info, ...","[jateng, jatim, jakarta, siaga, corona, info, ..."


In [51]:
# data testing
df_test['text_a'] = df_test['text_a'].apply(normalized_term)

In [52]:
df_test.head()

,text_a,label
0,"[jek, dajal, ga, depok, bang]",0
1,"[detikcom, untung, depok, masuk, wilayah, nya,...",0
2,"[df, dom, jakarta, depok, yg, gunain, vc, caba...",0
3,"[your, rl, depok, jkt]",0
4,"[doakan, indonesia, selamat, virus, corona, pk...",1


### **3.5 Stemming Bahasa Indonesia** 

In [22]:
#print(df_train['text_normal'].head(10))

In [23]:
#factory = StemmerFactory()
#stemmer = factory.create_stemmer()

In [24]:
#def stemmed_wrapper(term):
#    return stemmer.stem(term)

#term_dict = {}

#for document in df_train['text_normal']:
#    for term in document:
#        if term not in term_dict:
#           term_dict[term] = ' '

In [25]:
#print(len(term_dict))

In [26]:
#for term in term_dict:
#    term_dict[term] = stemmed_wrapper(term)
#    print(term,":" ,term_dict[term])

### **3.6 Input Preparation**

In [53]:
df_train.drop(columns = ['text_a', 'tokenized_text'], inplace = True)

In [54]:
def join_text_list(texts):
    #texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])

df_train["text_join"] = df_train["text_normal"].apply(join_text_list)

df_train.head()

,label,text_normal,text_join
2,1,"[ss, gini, buka, informasi, sejelas, nya, iden...",ss gini buka informasi sejelas nya identitas d...
7,1,"[jokowi, menteri, nya, silakan, tes, corona]",jokowi menteri nya silakan tes corona
8,1,"[pencegahan, corona, other, moms, minum, multi...",pencegahan corona other moms minum multivitami...
11,1,"[kemenkesri, mohon, yg, transparan, mudah, spt...",kemenkesri mohon yg transparan mudah spt utk d...
18,1,"[jateng, jatim, jakarta, siaga, corona, info, ...",jateng jatim jakarta siaga corona info disebar...


In [55]:
df_train.drop(columns = ['text_normal'], inplace = True)

In [56]:
# Data Testing 
df_test["text_a"] = df_test["text_a"].apply(join_text_list)
df_test.head()

,text_a,label
0,jek dajal ga depok bang,0
1,detikcom untung depok masuk wilayah nya ridwan...,0
2,df dom jakarta depok yg gunain vc cabang nya c...,0
3,your rl depok jkt,0
4,doakan indonesia selamat virus corona pkb depo...,1


## **Task 04 :**

Modelling

### **4.1 Split Data Training and Testing**

In [57]:
df_train.shape

(12178, 2)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(df_train['text_join'],df_train['label'], stratify=df_train['label'])

In [59]:
X_train.head()

7007     dedeags ustadtengkuzul adampena bos mpr pembua...
11027    lhokseumawe perintahkan dinas terkait petugas ...
6954     basmi hoax ujaran kebencian terciptanya pilkad...
4628              podoradong substansiny penyebaran corona
20120    terindikasi corona depok menkes difteri takut ...
Name: text_join, dtype: object

### **4.2 BERT processing**

In [60]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [61]:
# testing similarity

def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "sakit", 
    "tertular"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8737807 , -0.30251956, -0.02387387, ...,  0.00919212,
        -0.6210364 ,  0.8355628 ],
       [-0.8979806 , -0.19139743,  0.11907375, ..., -0.06637471,
        -0.5994425 ,  0.8543701 ]], dtype=float32)>

In [62]:
e = get_sentence_embeding([
    "sakit", 
    "sehat",
    "penyakit",
]
)

cosine_similarity([e[0]],[e[2]])

array([[0.8428337]], dtype=float32)

### **4.3 Create Model**

In [68]:
# BERT LAYER
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)


# NEURAL NETWORK LAYER
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


# FINAL MODEL
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [69]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                

### **4.4 Training Model**

In [70]:
len(X_train)

9133

In [71]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [72]:
model.fit(X_train, y_train, epochs=20, batch_size = 128)

Epoch 1/20
72/72 [==============================] - 237s 3s/step - loss: 0.6769 - accuracy: 0.5741 - precision: 0.5706 - recall: 0.5983
Epoch 2/20
72/72 [==============================] - 222s 3s/step - loss: 0.6559 - accuracy: 0.6121 - precision: 0.6092 - recall: 0.6248
Epoch 3/20
72/72 [==============================] - 223s 3s/step - loss: 0.6408 - accuracy: 0.6378 - precision: 0.6311 - recall: 0.6632
Epoch 4/20
72/72 [==============================] - 224s 3s/step - loss: 0.6279 - accuracy: 0.6542 - precision: 0.6482 - recall: 0.6743
Epoch 5/20
72/72 [==============================] - 225s 3s/step - loss: 0.6261 - accuracy: 0.6494 - precision: 0.6414 - recall: 0.6776
Epoch 6/20
72/72 [==============================] - 224s 3s/step - loss: 0.6158 - accuracy: 0.6666 - precision: 0.6611 - recall: 0.6835
Epoch 7/20
72/72 [==============================] - 222s 3s/step - loss: 0.6096 - accuracy: 0.6700 - precision: 0.6611 - recall: 0.6975
Epoch 8/20
72/72 [==============================

### **4.5 Model Evaluation**

In [73]:
model.evaluate(X_test, y_test)

96/96 [==============================] - 67s 686ms/step - loss: 0.5608 - accuracy: 0.7182 - precision: 0.7149 - recall: 0.7262


[0.5607748031616211,
 0.7182266116142273,
 0.7149321436882019,
 0.7261983156204224]

### **4.6 Model Prediction**

In [75]:
df_test.head()

,text_a,label
0,jek dajal ga depok bang,0
1,detikcom untung depok masuk wilayah nya ridwan...,0
2,df dom jakarta depok yg gunain vc cabang nya c...,0
3,your rl depok jkt,0
4,doakan indonesia selamat virus corona pkb depo...,1


In [76]:
y_pred = model.predict(df_test['text_a'])
output = np.where(y_pred>0.5,1, 0)
output

array([[0],
       [1],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [77]:
df_hasil = df_test
df_hasil['predicted'] = output
df_hasil.head()

,text_a,label,predicted
0,jek dajal ga depok bang,0,0
1,detikcom untung depok masuk wilayah nya ridwan...,0,1
2,df dom jakarta depok yg gunain vc cabang nya c...,0,1
3,your rl depok jkt,0,0
4,doakan indonesia selamat virus corona pkb depo...,1,1


In [78]:
df_hasil.tail()

,text_a,label,predicted
2795,ku tenang bae ku sih ya corona nya ga depok ku...,0,0
2796,guru hati hati ya virus corona uda indonesia t...,1,1
2797,terawan menyebut virus corona indonesia terdet...,1,1
2798,realffk buhari can pronounce corona virus,0,0
2799,hadapi wabah corona pemuda muhammadiyah pemeri...,1,1


In [79]:
print(classification_report(df_test['label'] , output))

              precision    recall  f1-score   support

           0       0.87      0.74      0.80      2093
           1       0.46      0.67      0.55       707

    accuracy                           0.72      2800
   macro avg       0.67      0.70      0.67      2800
weighted avg       0.77      0.72      0.73      2800

